In [1]:
from typing import List, Optional, Union, Tuple
from abstractions.goap.actions import Prerequisites, Goal
from abstractions.goap.entity import Statement, Attribute
from abstractions.goap.nodes import GameEntity, Node, Position
from abstractions.goap.language_state import GoalState
from abstractions.goap.gridmap import GridMap
from abstractions.goap.interactions import Character, Door, Key, Treasure, Floor, Wall, InanimateEntity, IsPickupable, TestItem, Open, Close, Unlock, Lock, Pickup, Drop, Move
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.shapes import Rectangle, Shadow, Radius, RayCast, BlockedRaycast
from abstractions.goap.agents import Agent

def source_target_position_comparison(source: tuple[int,int], target: tuple[int,int]) -> bool:
    """Check if the source entity's position is the same as the target entity's position."""
    if source and target:
        return source == target
    return False

def treasure_in_neighborhood(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the treasure is in the character's neighborhood."""
    if source.node and target and target.node:
        return source.node in target.node.neighbors()
    return False

def key_in_inventory(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the key is in the character's inventory."""
    if target:
        return target in source.inventory
    return False

def is_treasure(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the target entity is a Treasure."""
    return isinstance(target, Treasure)

def is_golden_key(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Golden Key."""
    return isinstance(target, Key) and target.key_name.value == "Golden Key"

def is_door(entity: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Door."""
    return isinstance(entity, Door)

# Example usage
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([Move, Pickup,  Open, Close, Unlock, Lock])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)
grid_map.generate_entity_type_map()

# Goal 1: Check if the character's position is the same as the treasure's position
reach_treasure_goal = Goal(
    name="Reach the treasure",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(comparisons={
                "source_position": ("position", "position", source_target_position_comparison)
            })
        ]
    )
)

# Goal 2: Check if the treasure is in the character's neighborhood
treasure_in_neighborhood_goal = Goal(
    name="Treasure in neighborhood",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(callables=[treasure_in_neighborhood])
        ]
    )
)

# Goal 3: Check if the key is in the character's inventory
key_in_inventory_goal = Goal(
    name="Key in inventory",
    source_entity_id=character.id,
    target_entity_id=key.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_golden_key])],
        source_target_statements=[
            Statement(callables=[key_in_inventory])
        ]
    )
)

# Goal 4: Check if the door is unlocked
door_unlocked_goal = Goal(
    name="Door unlocked",
    source_entity_id=door.id,
    prerequisites=Prerequisites(
        source_statements=[
            Statement(callables=[is_door]),
            Statement(conditions={"is_locked": False})
        ],
        target_statements=[],
        source_target_statements=[]
    )
)

# goals = [reach_treasure_goal, treasure_in_neighborhood_goal, key_in_inventory_goal, door_unlocked_goal]
goals = [key_in_inventory_goal]
model_path = "/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf"
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF/Nous-Hermes-2-Mixtral-8x7B-DPO.Q8_0.gguf"
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q8_0.gguf"
# model_path = None
agent = Agent(
    goals=goals,
    character_id=character.id,
    actions=grid_map.get_actions(),
    entity_type_map=grid_map.entity_type_map,
    llm = model_path

)
mdp = True
if mdp:
    shape = grid_map.get_rectangle()
else:
    character_node = GameEntity.get_instance(character.id).node
    shape = grid_map.get_shadow(source=character_node, max_radius=5)



........................................
llama_new_context_with_model: n_ctx      = 30016
llama_new_context_with_model: n_batch    = 512
llama_new_context_with_model: n_ubatch   = 512
llama_new_context_with_model: freq_base  = 1000000.0
llama_new_context_with_model: freq_scale = 1
ggml_metal_init: allocating
ggml_metal_init: found device: Apple M3 Max
ggml_metal_init: picking default device: Apple M3 Max
ggml_metal_init: using embedded metal library
ggml_metal_init: GPU name:   Apple M3 Max
ggml_metal_init: GPU family: MTLGPUFamilyApple9  (1009)
ggml_metal_init: GPU family: MTLGPUFamilyCommon3 (3003)
ggml_metal_init: GPU family: MTLGPUFamilyMetal3  (5001)
ggml_metal_init: simdgroup reduction support   = true
ggml_metal_init: simdgroup matrix mul. support = true
ggml_metal_init: hasUnifiedMemory              = true
ggml_metal_init: recommendedMaxWorkingSetSize  = 103079.22 MB
ggml_backend_metal_buffer_type_alloc_buffer: allocated buffer, size =  3752.00 MiB, (50949.80 / 98304.00)
llama_

In [2]:
agent.derive_allowed_strings(grid_map)

['East Move Floor',
 'North Move Floor',
 'NorthEast Move Floor',
 'NorthWest Move Floor',
 'West Move Floor']

In [3]:
agent.allowed_strings_count

{}

In [4]:
agent.run(grid_map, max_steps=10000, mdp=True)


--- Step 0 ---
Generating generator for ['East Move Floor', 'North Move Floor', 'NorthEast Move Floor', 'NorthWest Move Floor', 'West Move Floor']



llama_print_timings:        load time =    4684.39 ms
llama_print_timings:      sample time =       0.41 ms /     5 runs   (    0.08 ms per token, 12345.68 tokens per second)
llama_print_timings: prompt eval time =   17088.76 ms /  2663 tokens (    6.42 ms per token,   155.83 tokens per second)
llama_print_timings:        eval time =     205.96 ms /     4 runs   (   51.49 ms per token,    19.42 tokens per second)
llama_print_timings:       total time =   17308.72 ms /  2667 tokens



Action State:


Generated Action:
East Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='b68ec7cc-e25c-4864-a6ac-d71190537775' position=Position(name='Position', id='d8175c3b-69fc-4ba2-aa3f-74ab1f02a6b3', value=(6, 1)) entities=[Floor(id=b0bf2f17-5486-4a56-905b-66d6edec2d43, name=Floor_6_1, position=(6, 1))] gridmap_id='d5fb5c7d-efa2-4d8c-ac31-641303664ef8' blocks_movement=BlocksMovement(name='BlocksMovement', id='7e1081e4-6e96-4f40-bd8d-ef6d102728a2', value=False) blocks_light=BlocksLight(name='BlocksLight', id='a8d6113c-2366-419f-a29a-b52bdd1dd6d0', value=False) hash_resolution='default' starting from action string East Move Floor

Action Payload:
actions=[ActionInstance(source_id='f3fab05f-d9e4-4a59-9f29-e5f2cfb8c51d', target_id='b0bf2f17-5486-4a56-905b-66d6edec2d43', action=Move(name='Move Step', prerequisites=Prerequisites(source_statements=[Statement(name='Statement', id='8493120c-a519-40cd-8b0d-8eb9f2a53617', conditio


llama_print_timings:        load time =    4684.39 ms
llama_print_timings:      sample time =       0.51 ms /     7 runs   (    0.07 ms per token, 13725.49 tokens per second)
llama_print_timings: prompt eval time =   15339.62 ms /  2857 tokens (    5.37 ms per token,   186.25 tokens per second)
llama_print_timings:        eval time =     308.05 ms /     6 runs   (   51.34 ms per token,    19.48 tokens per second)
llama_print_timings:       total time =   15663.44 ms /  2863 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (5, 1) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_6_1' (6, 1) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (5, 1) -> (6, 1)]
### Natural Language Summary:
The position: (5, 1) of Character 'Player' changed to (6, 1) as a result of Move.


Generated Action:
NorthEast Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='c2b81a54-44d1-4c0d-b856-e58989f71e7f' position=Position(name='Position', id='43cb38dd-f967-4c34-8a29-40f645d6f533', value=(7, 0)) entities=[Floor(id=1450f95e-3725-43d5-b607-545060a6357f, name=Floor_7_0, position=(7, 0))] gridmap_id='d5fb5c7d-efa2-4d8c-ac31-641303664ef8' blocks_movement=BlocksMovement(name='BlocksMoveme

...


Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (8, 1) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_9_2' (9, 2) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (8, 1) -> (9, 2)]
### Natural Language Summary:
The position: (8, 1) of Character 'Player' changed to (9, 2) as a result of Move.


Generated Action:
West Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='c640f058-f637-4936-b7d7-06b61e25725d' position=Position(name='Position', id='eedaf53e-f97e-444f-ac8d-6540f66efb61', value=(8, 2)) entities=[Floor(id=6a8a27af-ef7f-48a6-97ca-2222875eb5fe, name=Floor_8_2, position=(8, 2))] gridmap_id='d5fb5c7d-efa2-4d8c-ac31-641303664ef8' blocks_movement=BlocksMovement(name='BlocksMovement', 


llama_print_timings:        load time =    4684.39 ms
llama_print_timings:      sample time =       0.61 ms /     8 runs   (    0.08 ms per token, 13114.75 tokens per second)
llama_print_timings: prompt eval time =   20541.69 ms /  3566 tokens (    5.76 ms per token,   173.60 tokens per second)
llama_print_timings:        eval time =     374.82 ms /     7 runs   (   53.55 ms per token,    18.68 tokens per second)
llama_print_timings:       total time =   20937.27 ms /  3573 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (9, 2) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_8_2' (8, 2) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (9, 2) -> (8, 2)]
### Natural Language Summary:
The position: (9, 2) of Character 'Player' changed to (8, 2) as a result of Move.


Generated Action:
SouthEast Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='755fb62f-485b-4baa-a94d-30f2a32b4278' position=Position(name='Position', id='6dae93f4-2f98-40a8-9252-868f04e9f33a', value=(9, 3)) entities=[Floor(id=d2e9695d-359c-4e0e-bfe7-88d08a847f7d, name=Floor_9_3, position=(9, 3))] gridmap_id='d5fb5c7d-efa2-4d8c-ac31-641303664ef8' blocks_movement=BlocksMovement(name='BlocksMoveme


llama_print_timings:        load time =    4684.39 ms
llama_print_timings:      sample time =       0.31 ms /     4 runs   (    0.08 ms per token, 12903.23 tokens per second)
llama_print_timings: prompt eval time =   20378.26 ms /  3682 tokens (    5.53 ms per token,   180.68 tokens per second)
llama_print_timings:        eval time =     164.05 ms /     3 runs   (   54.68 ms per token,    18.29 tokens per second)
llama_print_timings:       total time =   20557.13 ms /  3685 tokens



Action State:
# Action Result Analysis
## Action: Move
### Result: Success
### Before:
- Source: Character 'Player' (8, 2) [blocks_movement: False, blocks_light: False, health: 100, max_health: 100, attack_power: 10, can_act: True, inventory: []]
- Target: Floor 'Floor_9_3' (9, 3) [blocks_movement: False, blocks_light: False, material: , inventory: []]
### Changes:
- Source: Character 'Player' [position: (8, 2) -> (9, 3)]
### Natural Language Summary:
The position: (8, 2) of Character 'Player' changed to (9, 3) as a result of Move.


Generated Action:
West Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='d932a91e-8fe4-4c34-974b-2ccc8e0d7dd4' position=Position(name='Position', id='bea0f03e-93da-43b3-bbb7-781224f4bbb7', value=(8, 3)) entities=[Floor(id=fb070339-99eb-4773-a24e-e9a77b69e840, name=Floor_8_3, position=(8, 3))] gridmap_id='d5fb5c7d-efa2-4d8c-ac31-641303664ef8' blocks_movement=BlocksMovement(name='BlocksMovement', 

In [ ]:
# obs_text, act_text, goal_text = agent.generate_text(shape)

In [ ]:
# agent.generate_action(obs_text, act_text, goal_text)

In [ ]:
# agent = Agent(
#     goals=goals,
#     character_id=character.id,
#     actions=grid_map.get_actions(),
#     entity_type_map=grid_map.entity_type_map
# )

# # Run the agent
# agent.run(grid_map, max_steps=10000, mdp=True)

In [ ]:
# gang = Character.get_instance(character.id)
